In [1]:
import requests
from pprint import pprint
import pandas as pd
import re


lat = 59.3293
lon = 18.0686

def get_smhi_forecast(lat, lon):
    url = "https://opendata-download-metobs.smhi.se/api.json"
    r = requests.get(url)
    r.raise_for_status()
    return r.json()

data = get_smhi_forecast(lat, lon)
pprint(data)

{'key': 'metobs',
 'link': [{'href': 'https://opendata-download-metobs.smhi.se/api.json',
           'rel': 'category',
           'type': 'application/json'},
          {'href': 'https://opendata-download-metobs.smhi.se/api.xml',
           'rel': 'category',
           'type': 'application/xml'},
          {'href': 'https://opendata-download-metobs.smhi.se/api.atom',
           'rel': 'category',
           'type': 'application/atom+xml'}],
 'summary': 'Nedladdningstjänst för svenska meteorologiska observationsdata '
            'från SMHI (enligt krav från INSPIRE-direktivet). Du kan ladda ned '
            'data för en parameter och en station i taget. Erbjudna format är '
            'Atomflöde, XML och JSON. Nedladdningstjänsten är strukturerad i '
            'följande nivåer: (1) Parameter (2) Station (3) Tidsutsnitt enligt '
            'URL-strukturen '
            'https://opendata-download-metobs-utv.smhi.se/api/version/{version}/parameter/{parameter}/station/{station}/peri

In [5]:
version   = "latest"
parameter = 11 # Global Irradians (svenska stationer)	medelvärde 1 timma, varje timme	watt per kvadratmeter
station   = 98735
period    = "latest-months"
data      = "json"
url =     f"https://opendata-download-metobs.smhi.se/api/version/{version}/parameter/{parameter}/station/{station}/period/{period}/data.{data}"

response = requests.get(url)
response.raise_for_status()


pprint(response)

text = response.json()

pprint(text)


<Response [200]>
{'link': [{'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/11/station/98735/period/latest-months/data.json',
           'rel': 'data',
           'type': 'application/json'},
          {'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/11/station/98735/period/latest-months/data.xml',
           'rel': 'data',
           'type': 'application/xml'},
          {'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/11/station/98735/period/latest-months/data.csv',
           'rel': 'data',
           'type': 'text/plain'},
          {'href': 'https://opendata-download-metobs.smhi.se/api/version/1.0/parameter/11/station/98735/period/latest-months.atom',
           'rel': 'period',
           'type': 'application/atom+xml'},
          {'href': 'https://opendata-catalog.smhi.se/md/25080190-38ba-4279-a65d-d9ef8d0bf949',
           'rel': 'iso19139',
           'type': 'application/vnd.iso.19139+xml'},
 

In [6]:
import pandas as pd
from datetime import datetime, timezone

# Extract the list under "value"
rows = text["value"]

df = pd.DataFrame(rows)

# Convert SMHI timestamp (ms → seconds → datetime)
df["timestamp"] = pd.to_datetime(df["date"], unit="ms", utc=True)

# Convert value column from string to float
df["value"] = df["value"].astype(float)

# Reorder columns
df = df[["timestamp", "value", "quality"]]

print(df)

                     timestamp  value quality
0    2025-07-31 01:00:00+00:00   0.00       G
1    2025-07-31 02:00:00+00:00   0.00       G
2    2025-07-31 03:00:00+00:00   0.33       G
3    2025-07-31 04:00:00+00:00   5.00       G
4    2025-07-31 05:00:00+00:00  34.23       G
...                        ...    ...     ...
3136 2025-12-08 17:00:00+00:00   0.37       Y
3137 2025-12-08 18:00:00+00:00   0.30       Y
3138 2025-12-08 19:00:00+00:00   0.16       Y
3139 2025-12-08 20:00:00+00:00   0.15       Y
3140 2025-12-08 21:00:00+00:00   0.12       Y

[3141 rows x 3 columns]
